In [ ]:
!pip install langchain langchain-community langchain-core pypdf faiss-cpu sentence-transformers langchain_google_genai langchain_teddynote langchain_openai langchain_chroma

# 모델 불러오기

In [ ]:
import os
from google.colab import userdata # 코랩 Secrets 기능 사용을 위한 임포트
from langchain_google_genai import ChatGoogleGenerativeAI

# 코랩 Secrets에서 GOOGLE_API_KEY를 가져오기
try:
    google_api_key = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = google_api_key
    print("GOOGLE_API_KEY가 환경 변수로 설정되었습니다.")
except userdata.SecretNotFoundError:
    print("오류: 'GOOGLE_API_KEY' 비밀을 코랩 Secrets에 설정해주세요. ")
    exit() # API 키가 없으면 스크립트 종료
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")
print(f"Selected LLM: {llm.__class__.__name__}")

GOOGLE_API_KEY가 환경 변수로 설정되었습니다.
Selected LLM: ChatGoogleGenerativeAI


#프롬프트

## 변수 하나

In [ ]:
from langchain_core.prompts import PromptTemplate
# template 정의.
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"] #변수 하나일 땐 생략 가능
)

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [ ]:
# prompt 생성. format 메소드를 이용하여 변수에 값을 넣어줌
prompt = prompt.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

### 결과 출력

In [ ]:
#결과 출력_체인 생성x
from langchain_teddynote.messages import stream_response
#stream 이용하여 실시간으로 확인. 체인 생성x
answer = llm.stream(prompt)
stream_response(answer)

대한민국의 수도는 **서울**입니다.


In [ ]:
#결과 출력_체인 생성x
#한번에 출력
answer = llm.invoke(prompt)
print(answer.content)

대한민국의 수도는 **서울**입니다.


In [ ]:
#결과 출력_체인 생성

# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

#chain 생성
chain = prompt | llm

# 체인, 실시간 출력
answer = chain.stream(
    {"country": "대한민국"}
)
stream_response(answer)


대한민국의 수도는 **서울**입니다.


In [ ]:
#결과 출력_체인 생성

#한번에 출력
chain.invoke("대한민국").content

'대한민국의 수도는 **서울**입니다.'

## 변수 두 개

In [ ]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [ ]:
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

In [ ]:
# chain 생성
chain = prompt | llm
print(chain.invoke('대한민국'))
print(chain.invoke('대한민국').content)

content='대한민국의 수도는 **서울**이고, 미국의 수도는 **워싱턴 D.C.**입니다.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash-latest', 'safety_ratings': []} id='run--92908044-d512-4239-9b46-2ce6f1e785df-0' usage_metadata={'input_tokens': 17, 'output_tokens': 27, 'total_tokens': 44, 'input_token_details': {'cache_read': 0}}
대한민국의 수도는 **서울**이고, 미국의 수도는 **워싱턴 D.C.**입니다.


In [ ]:
prompt_partial = prompt.partial(country2="캐나다")
print(prompt_partial.format(country1="대한민국"))
print(chain.invoke('대한민국').content) #chain에 사용된 것이 prompt_partial이 아닌 prompt이므로 country2는 미국.

대한민국과 캐나다의 수도는 각각 어디인가요?
대한민국의 수도는 **서울**이고, 미국의 수도는 **워싱턴 D.C.**입니다.


In [ ]:
chain = prompt_partial | llm
print(chain.invoke('대한민국').content)

대한민국의 수도는 **서울**이고, 캐나다의 수도는 **오타와**입니다.


## 상세한 지시

In [ ]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 입력된 상황에 사용될 회화를 [FORMAT]에 맞추어 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

chain = prompt | llm
print(chain.invoke('식당에 가서 주문하기').content)

- 영어 회화:

Waiter: Hello, welcome to [Restaurant Name].  Can I help you?
Me: Yes, please.  We'd like a table for two.
Waiter: Certainly.  Follow me, please.  (Waiter leads to a table)
Waiter: Here you are.  Can I get you some drinks while you look at the menu?
Me: Yes, please.  We'll have two glasses of water.
Waiter:  Okay, two glasses of water.  I'll be back to take your order in a few minutes.
(After looking at the menu)
Me: Excuse me.  We're ready to order now.
Waiter:  Certainly.  What can I get for you?
Me: I'll have the [Dish Name], please.
Partner: And I'll have the [Dish Name].
Waiter:  Okay, one [Dish Name] and one [Dish Name].  Would you like anything else?
Me:  No, thank you. That's all.
Waiter:  Okay, I'll be right back with your drinks and food will be about [Time] minutes.
(After the meal)
Waiter:  How was everything?
Me:  It was delicious, thank you.
Waiter:  Great!  Can I get you anything else?  Dessert, coffee?
Me: No, thank you. We're finished.  Can we have the bill, 

# 퓨샷 프롬프트

In [ ]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]


In [ ]:
# ** 연산자

def greet(name, age):
    print(f"안녕하세요, {name}님. 나이는 {age}세시군요.")

greet(name="김철수", age=30)
# 출력: 안녕하세요, 김철수님. 나이는 30세시군요.

user_info = {
    "name": "김철수",
    "age": 30
}

# 딕셔너리를 키워드 인자(**)로 풀어서 전달
greet(**user_info)
# 출력: 안녕하세요, 김철수님. 나이는 30세시군요.

안녕하세요, 김철수님. 나이는 30세시군요.
안녕하세요, 김철수님. 나이는 30세시군요.


In [ ]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

#첫번째 Quetion, Answer 출력
print(example_prompt.format(**examples[0]))


Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인



In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)


Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인


Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question:
율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군


Question:
올드보이와 기생충의 감독이 같은 나라 출신인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예


Question:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
Answer:


In [ ]:
from langchain_teddynote.messages import stream_response
# 결과 출력
answer = llm.stream(final_prompt)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년에 태어났습니다.
추가 질문: 1998년에 Bill Gates의 나이는 몇 살이었나요?
중간 답변: 1998년에 Bill Gates는 43세였습니다.
최종 답변은: 43세


## example selector 사용

In [ ]:
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("OPENAI_API_KEY가 환경 변수로 설정되었습니다.")
except userdata.SecretNotFoundError:
    print("오류: 'OPENAI_API_KEY' 비밀을 코랩 Secrets에 설정해주세요. ")
    print("설정 방법: 왼쪽 사이드바의 열쇠 모양 아이콘 클릭 -> 'OPENAI_API_KEY' 이름으로 키 저장.")
    exit() # API 키가 없으면 스크립트 종료

OPENAI_API_KEY가 환경 변수로 설정되었습니다.


In [ ]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Vector DB 생성 (저장소 이름, 임베딩 클래스)
chroma = Chroma("example_selector", OpenAIEmbeddings())

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 선택 가능한 예시 목록.
    examples,
    # 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스
    OpenAIEmbeddings(),
    # 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스
    Chroma,
    # 생성할 예시의 수
    k=1,
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
# 입력과 가장 유사한 예시를 선택합니다.
selected_examples = example_selector.select_examples({"question": question})


print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')


입력에 가장 유사한 예시:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?

question:
네이버의 창립자는 언제 태어났나요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일



In [ ]:
#prompt 생성
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)
question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
example_selector_prompt = prompt.format(question=question)
print(example_selector_prompt)

Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
Answer:


In [ ]:
from langchain_teddynote.messages import stream_response
#결과 출력
answer = llm.stream(example_selector_prompt)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.

추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.

추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.

추가 질문: 1998년에 Bill Gates의 나이는 몇 살이었나요?
중간 답변: 1998년은 1955년부터 43년 후이므로, Bill Gates는 1998년에 42살이었습니다.

최종 답변은: 42살


## FewShotChatMessagePromptTemplate


*   (세부 지시 사항,) 사용자 입력, 그에 따른 답변 예시 준비
*   예시 프롬프트(사용자 입력, 답변) -> 퓨샷 프롬프트(예시 전체, 혹은 example selector) -> 최종 프롬프트(시스템 지시, 퓨샷 프롬프트)



In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "제품이 너무 빨리 배송되어서 놀랐어요! 정말 만족합니다.",
        "output": "긍정"
    },
    {
        "input": "배송이 늦어서 아쉬웠습니다. 다음엔 개선 부탁드려요.",
        "output": "부정"
    },
    {
        "input": "고객 서비스가 정말 친절해서 기분이 좋았어요!",
        "output": "긍정"
    },
]

# 2. 각 퓨샷 예시를 포맷팅할 ChatPromptTemplate 정의
# 이 템플릿은 각 예시의 input과 output이 HumanMessage와 AIMessage로 매핑되는 방식을 정의합니다.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# 3. FewShotChatMessagePromptTemplate 생성
# 위에서 정의한 example_prompt와 examples를 사용하여 퓨샷 프롬프트를 구성합니다.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt, # 각 예시의 형식
    examples=examples,            # 퓨샷 예시 데이터
    # optional: example_selector=your_example_selector,
)

# 4. 최종 ChatPromptTemplate 구성
# 시스템 메시지, 퓨샷 예시, 그리고 실제 사용자 질문을 결합합니다.
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "주어진 고객 문의의 감정을 긍정 또는 부정으로 분류하세요."),
        few_shot_prompt, # 여기에 퓨샷 예시 메시지들이 삽입됩니다.
        ("human", "{user_input}"), # 실제 사용자 질문
    ]
)

# 5. 체인 생성 및 실행
chain = final_prompt | llm

# 사용자 입력
user_question = "환불 절차가 너무 복잡해서 불만입니다."

# 체인 호출
response = chain.invoke({"user_input": user_question})
print(response.content)

부정


In [ ]:
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
   - 회의는 김수진 팀장의 개회사로 시작됨.
   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

2. 시장 동향 개요 (김수진)
   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

3. 디지털 마케팅 전략 (박지민)
   - 박지민은 디지털 마케팅 전략에 대해 발표.
   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

4. 소셜 미디어 캠페인 (이준호)
   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

5. 종합 논의
   - 팀원들 간의 아이디어 공유 및 토론.
   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 박지민 담당.
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

chroma = Chroma("fewshot_chat", OpenAIEmbeddings())

#예시 프롬프트 생성(human-ai)
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 선택 가능한 예시 목록.
    examples,
    # 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스
    OpenAIEmbeddings(),
    # 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스
    Chroma,
    # 생성할 예시의 수
    k=1,
)

#퓨샷 프롬프트 생성(example selector, 예시 프롬프트)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
)

In [ ]:
#가장 유사한 예시 하나 출력
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

example_selector.select_examples(question)

[{'answer': '\n회의록: XYZ 회사 마케팅 전략 회의\n일시: 2023년 12월 25일\n장소: XYZ 회사 회의실\n참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n1. 개회\n   - 회의는 김수진 팀장의 개회사로 시작됨.\n   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n2. 시장 동향 개요 (김수진)\n   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n3. 디지털 마케팅 전략 (박지민)\n   - 박지민은 디지털 마케팅 전략에 대해 발표.\n   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n4. 소셜 미디어 캠페인 (이준호)\n   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n5. 종합 논의\n   - 팀원들 간의 아이디어 공유 및 토론.\n   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.\n\n6. 마무리\n   - 다음 회의 날짜 및 시간 확정.\n   - 회의록 정리 및 배포는 박지민 담당.\n',
  'input': '2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.',
  'instruction': '당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요'}]

In [ ]:
#최종 프롬프트(시스템 지시, 퓨샷 프롬프트, 사용자 입력)
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        few_shot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)
# chain 생성
chain = final_prompt | llm

# 실행 및 결과 출력
answer = chain.stream(question)
stream_response(answer)


**회의록**

**일시:** 2023년 12월 26일

**장소:** ABC 기술 회사 회의실

**참석자:** 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

**주제:** 새로운 모바일 애플리케이션 프로젝트 주간 진행 상황 회의

**1. 개회 (최현수)**

* 최현수 프로젝트 매니저가 회의를 시작하며, 이번 회의의 목적이 프로젝트 진행 상황 검토 및 다음 주 목표 설정임을 명시함.


**2. 진행 상황 보고**

* **황지연 (주요 개발자):**
    * 백엔드 개발 진행 상황 보고: API 개발 80% 완료, 데이터베이스 설계 완료.  다음 주에는 API 테스트 및 버그 수정에 집중할 예정.  예상보다 약간의 지연이 발생했으며, 그 원인과 해결 방안에 대해 설명.
    * 현재 직면한 기술적 문제점: 특정 라이브러리 호환성 문제. 해결 방안 모색 중.
* **김태영 (UI/UX 디자이너):**
    * UI 디자인 초안 완료 및 개발팀에 전달.  다음 주에는 사용자 피드백을 바탕으로 디자인 수정 및 개선 작업 진행 예정.  주요 화면 디자인에 대한 시각 자료를 제시하고 설명.
* **최현수 (프로젝트 매니저):**
    * 전체적인 프로젝트 일정 및 예산 현황 보고.  현재까지는 예상 일정 내에 진행되고 있으나, 지연 가능성을 고려하여 예비 계획 수립 필요성 제기.


**3. 다음 주 목표 설정**

* **황지연:** API 테스트 완료 및 버그 수정 90%, 기술적 문제 해결.
* **김태영:** UI 디자인 최종 수정 완료 및 개발팀과의 협업 강화.
* **최현수:**  프로젝트 위험 요소 분석 및 대응 방안 마련, 다음 주 진행 상황 회의 준비.


**4. 기타 논의 사항**

* 다음 주 회의 시간 조정에 대한 논의 및 결정. (시간: 2023년 12월 27일 오전 10시)
* 외부 라이브러리 사용에 대한 추가적인 검토 필요성 제기.


**5. 폐회**

* 최현수 프로젝트 매니저가 회의

## 커스텀 예제 선택기




In [ ]:
# input을 입력하지 않고 instruction만 입력 시 결과가 제대로 나오지 않음
question = {
    "instruction": "회의록을 작성해 주세요",
}

example_selector.select_examples(question)


[{'answer': '본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.',
  'instruction': '당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요',
  'input': '우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.'}]

In [ ]:
from langchain_teddynote.prompts import CustomExampleSelector

# 커스텀 예제 선택기 생성
custom_selector = CustomExampleSelector(examples, OpenAIEmbeddings())

# 커스텀 예제 선택기를 사용했을 때 결과
custom_selector.select_examples({"instruction": "회의록을 작성해 주세요"})


[{'instruction': '당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요',
  'input': '2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.',
  'answer': '\n회의록: XYZ 회사 마케팅 전략 회의\n일시: 2023년 12월 25일\n장소: XYZ 회사 회의실\n참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n1. 개회\n   - 회의는 김수진 팀장의 개회사로 시작됨.\n   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n2. 시장 동향 개요 (김수진)\n   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n3. 디지털 마케팅 전략 (박지민)\n   - 박지민은 디지털 마케팅 전략에 대해 발표.\n   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n4. 소셜 미디어 캠페인 (이준호)\n   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n5. 종합 논의\n   - 팀원들 간의 아이디어 공유 및 토론.\n   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.\n\n6. 마무리\n   - 다음 회의 날짜 및 시간 확정.\n   - 회의록 정리 및 배포는 박지민 담당.\n'}]

In [ ]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

custom_fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=custom_selector,  # 커스텀 예제 선택기 사용
    example_prompt=example_prompt,  # 예제 프롬프트 사용
)

custom_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        custom_fewshot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)


In [ ]:
# chain 생성
chain = custom_prompt | llm
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))


**회의록**

**일시:** 2023년 12월 26일

**주제:** 새로운 모바일 애플리케이션 프로젝트 주간 진행 상황 회의

**참석자:** 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

**1. 개회 (최현수)**

* 최현수 프로젝트 매니저가 회의를 시작하며, 목적을 밝힘: 프로젝트 진행 상황 검토 및 다음 주 목표 설정.

**2. 진행 상황 보고**

* **황지연 (주요 개발자):**
    * 백엔드 개발 진행 상황 보고: API 개발 80% 완료, 예상보다 2일 지연. 지연 이유는 예상치 못한 데이터베이스 이슈 발생. 해결 방안 모색 중.
    * 다음 주 목표: API 개발 완료 및 테스트 시작. 데이터베이스 이슈 완벽 해결.
* **김태영 (UI/UX 디자이너):**
    * UI 디자인 완료, UX 프로토타입 개발 90% 완료. 사용자 테스트 진행 예정.
    * 다음 주 목표: UX 프로토타입 최종 완료 및 사용자 테스트 실시. 디자인 가이드라인 문서 작성 완료.
* **최현수 (프로젝트 매니저):**
    * 전체 프로젝트 일정 관리 및 리스크 관리.
    * 백엔드 개발 지연에 대한 대응 방안 논의. 일정 조정 가능성 검토.
    * 다음 주 목표: 개발 지연에 대한 리스크 평가 및 대응 계획 수립.  다음 주 회의를 위한 진행 상황 보고서 작성.

**3. 문제점 및 해결 방안**

* 백엔드 개발 지연 문제:  황지연 개발자의 추가 지원 필요.  최현수 프로젝트 매니저가 다른 개발자 지원 가능성 검토.
* 사용자 테스트 결과에 따른 디자인 수정: 김태영 디자이너의 유연한 대응 필요.

**4. 다음 주 목표**

* 백엔드 API 개발 완료 및 테스트 시작 (황지연)
* UX 프로토타입 최종 완료 및 사용자 테스트 실시 (김태영)
* 개발 지연 리스크 평가 및 대응 계획 수립 (최현수)
* 다음 주 회의 전까지 진행 상황 보고서 제출 (모든 참석자)


**5. 폐회 (최현수)*

# 랭체인 허브

In [ ]:
from langchain import hub

# 가장 최신 버전의 프롬프트를 가져옵니다.
prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
# 프롬프트 내용 출력
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [ ]:
prompt = ChatPromptTemplate.from_template(
    "주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: {context}\n\nSUMMARY:"
)
prompt

ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: {context}\n\nSUMMARY:'), additional_kwargs={})])

In [ ]:
from langchain import hub

# 프롬프트를 허브에 업로드합니다.
#hub.push("id/simple-summary-korean", prompt)

# 출력 파서


*   get_format_instruction(): 언어 모델이 출력해야 할 정보의 형식 정의



## PydanticOutputParser

*   사용자가 원하는 정보를 명확하게 제공



In [ ]:
from langchain_teddynote.messages import stream_response
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [ ]:
#이메일 본문 예시
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""


In [ ]:
#출력 파서 이용x
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)


이메일의 중요한 내용은 다음과 같습니다:

1. 김철수 상무는 바이크코퍼레이션 소속이며, 이은채 대리에게 이메일을 보냈습니다.
2. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 활동하는 기업입니다.
3. 김철수 상무는 "ZENESIS" 자전거에 대한 상세한 브로슈어를 요청하며, 특히 기술 사양, 배터리 성능, 디자인 측면의 정보를 필요로 합니다.
4. 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안했습니다.
5. 미팅 장소는 이은채 대리가 근무하는 회사의 사무실입니다.

In [ ]:
#description은 추출할 정보 설명
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [ ]:
# instruction 출력
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


### 방법1- 체인에 프롬프트, llm 연결. 결과를 직접 반환

In [ ]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())


In [ ]:
# chain 을 생성
chain = prompt | llm
# 결과 출력
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됨
output = stream_response(response, return_output=True)


```json
{
    "person": "김철수",
    "email": "chulsoo.kim@bikecorporation.me",
    "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
    "summary": "김철수 상무는 바이크코퍼레이션에서 ZENESIS 자전거의 브로슈어를 요청하며, 기술 사양, 배터리 성능, 디자인 정보가 필요하다고 언급했습니다. 또한, 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안했습니다.",
    "date": "1월 15일 화요일 오전 10시"
}
```

In [ ]:
# PydanticOutputParser 를 사용하여 결과 파싱. EmailSummary 객체 형태
structured_output = parser.parse(output)
structured_output

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 브로슈어를 요청하고, 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안합니다.', date='1월 15일 화요일 오전 10시')

### 방법2- 체인에 프롬프트, llm, 파서 연결

In [ ]:
# 출력 파서를 추가하여 전체 체인 재구성
chain = prompt | llm | parser

# 결과 출력
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력
response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 브로슈어를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안합니다.', date='1월 15일 화요일 오전 10시')

### 방법3- llm 호출할 때 .with_structured_output 이용

In [ ]:
#.with_structured_output 이용하여 출력을 pydantic 객체로 변환
llm_with_structered = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest").with_structured_output(EmailSummary)

In [ ]:
# invoke() 함수를 호출하여 결과 출력 (stream 불가)
answer = llm_with_structered.invoke(email_conversation)
answer

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션 김철수 상무는 TEDDY INTERNATIONAL의 신규 자전거 "ZENESIS"에 대한 유통 협력을 제안하며, 제품 브로슈어 요청 및 1월 15일 화요일 오전 10시 미팅을 제안했다.', date='1월 15일 오전 10시')

## CommaSeparatedListOutputParser

*   콤마로 구분된 리스트 출력 파서



In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()
# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    # 주제에 대한 다섯 가지를 나열하라는 템플릿
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],  # 입력 변수로 'subject' 사용
    # 부분 변수로 형식 지침 사용
    partial_variables={"format_instructions": format_instructions},
)

# ChatOpenAI 모델 초기화
##temperature=0은 동일한 입력에 동일한 출력이 최대한 나오도록 함. 높을수록 창의적, 낮을수록 일관적
### 2로 설정 하였을 때 결과: ['경복궁', '남산한옥마을', '설악산', '제주도', '광화문'] / ['경복궁', '남산타워', '설악산', '제주도', '부산 해운대']
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser


In [ ]:
# "대한민국 관광명소"에 대한 체인 호출 실행
chain.invoke({"subject": "대한민국 관광명소"})

['경복궁', '남산타워', '설악산', '제주도', '광화문']

In [ ]:
# 스트림 순회
for s in chain.stream({"subject": "대한민국 관광명소"}):
    print(s)  # 스트림의 내용을 출력

['경복궁']
['남산타워']
['설악산']
['제주도']
['광화문']


## StructuredOuputParser


*   구조화된 출력 파서
*   LLM에 대한 답변을 dict 형식으로 구성하고 key/value 쌍으로 갖는 여러 필드를 반환하고자 할 때 사용
*   Pydantic/JSON 파서가 더 강력. 대신 인텔리전스가 낮은(파라미터 수가 적은) 모델에는 StructuredOuputParser가 더 적합(대안으로 사용 가능).

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
# 응답 스키마. 사용자의 질문에 대한 답변 구조화
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]
# 응답 스키마를 기반으로 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
# 출력 형식 지시사항 파싱
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿 설정
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question' 사용
    input_variables=["question"],
    # 부분 변수로 'format_instructions' 사용
    partial_variables={"format_instructions": format_instructions},
)


In [ ]:
chain = prompt | model | output_parser  # 프롬프트, 모델, 출력 파서를 연결
# 앞서 정의한 스키마대로 출력
chain.invoke({"question": "대한민국의 수도는 어디인가요?"})

{'answer': '대한민국의 수도는 서울입니다.',
 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8'}

In [ ]:
for s in chain.stream({"question": "세종대왕의 업적은 무엇인가요?"}):
    # 스트리밍 출력
    print(s)

{'answer': '세종대왕의 업적은 매우 다양하고 광범위하지만, 크게 다음과 같이 정리할 수 있습니다.\n\n**1. 훈민정음 창제:** 가장 큰 업적으로 꼽히는 것은 백성을 위한 언어 창제입니다. 훈민정음은 과학적인 원리에 기반한 독창적인 문자로, 한글의 우수성은 세계적으로 인정받고 있습니다. 이는 문맹률 감소와 문화 발전에 크게 기여했습니다.\n\n**2. 과학기술 발전:** 세종대왕 시대는 과학기술이 눈부시게 발전한 시기였습니다.  장영실 등 뛰어난 과학자들을 등용하여 측우기, 자격루, 앙부일구 등 다양한 과학 기구를 발명하고, 농업 기술을 발전시켰습니다. 이는 국가의 경제력 향상과 국민 생활 수준 향상에 기여했습니다.\n\n**3. 군사력 강화:**  세종대왕은 국방력 강화에도 힘썼습니다. 거북선을 비롯한 새로운 무기를 개발하고, 군사 제도를 정비하여 국토 방위를 튼튼히 했습니다.  이를 통해 잦은 외침으로부터 나라를 안전하게 지킬 수 있었습니다.\n\n**4. 정치 개혁:**  세종대왕은 효율적인 정치 시스템 구축을 위해 노력했습니다.  각종 법률과 제도를 정비하고, 관리들의 윤리 의식을 강화하여 부정부패를 줄이고 공정한 사회를 만들고자 했습니다.\n\n**5. 문화 발전:**  세종 시대는 예술과 문화가 크게 발전한 시기이기도 합니다.  음악, 그림, 문학 등 다양한 분야에서 뛰어난 작품들이 많이 만들어졌습니다.  이러한 문화적 성과는 국가의 위상을 높이고 국민들의 정신적 풍요를 가져왔습니다.\n\n이 외에도 세종대왕은 농업 생산 증대, 교육 제도 개선, 외교 관계 개선 등 다방면에서 뛰어난 업적을 남겼습니다.  세종대왕의 업적은 단순히 개인의 업적을 넘어, 한국 역사와 문화 발전에 지대한 영향을 미쳤다는 점에서 그 의미가 매우 큽니다.', 'source': '다양한 한국사 교과서 및 참고서, 한국민족문화대백과사전 웹사이트 (www.encykorea.aks.ac.kr)'}


## JsonOutputParser

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [ ]:
# 원하는 데이터 구조 정의
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [ ]:
#pydantic 이용. Pydantic 객체가 요구하는 정확한 JSON 스키마를 포함-강제성 높음.


# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입
parser = JsonOutputParser(pydantic_object=Topic)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser  # 체인 구성

chain.invoke({"question": question})  # 체인을 호출하여 쿼리 실행


{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 인간 활동으로 인한 온실가스 배출 증가가 원인입니다. 이로 인해 극심한 기상 현상, 해수면 상승, 생태계 교란 등 심각한 문제가 발생합니다.',
 'hashtags': '#지구온난화 #기후변화 #온실가스 #환경문제 #지구온난화심각성'}

In [ ]:
#pydantic 이용x. 자유성 높아짐.


# 질의 작성
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력 확인
## Topic 클래스에서 해시태그의 자료형을 문자열로 정의했으나, pydantic로 강제하지 않았으므로 llm의 선택에 따라 리스트 형식으로 출력됨.
print(response)


{'description': '지구 온난화는 지구의 평균 기온이 장기간에 걸쳐 상승하는 현상입니다. 주요 원인은 인간 활동으로 인한 온실가스 배출 증가로, 이는 대기 중 열을 가두어 지구 온도를 높입니다. 이로 인해 해수면 상승, 극심한 기상 현상, 생태계 교란 등 심각한 문제가 발생합니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경오염', '#지구온도상승', '#기후위기', '#탄소배출', '#지속가능한발전', '#녹색성장', '#환경보호']}


## PandasDataFrameOutputParser

In [ ]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
# 출력 목적으로만 사용
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    # 파서 출력의 키들을 순회
    for key in parser_output.keys():
        # 각 키의 값을 딕셔너리로 변환
        parser_output[key] = parser_output[key].to_dict()
    # 예쁘게 출력
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

In [ ]:
import pandas as pd
# 원하는 Pandas DataFrame 정의
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# 파서를 설정하고 데이터 프레임 입력
parser = PandasDataFrameOutputParser(dataframe=df)

# 파서의 지시사항 출력
print(parser.get_format_instructions())


The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [ ]:
# 열 작업 예시
df_query = "Age column 을 조회해 주세요."


# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # 입력 변수 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # 부분 변수 설정
)

# 체인 생성
chain = prompt | model | parser

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 출력
format_parser_output(parser_output)

{'Age': {0: 22.0,
         1: 38.0,
         2: 26.0,
         3: 35.0,
         4: 35.0,
         5: nan,
         6: 54.0,
         7: 2.0,
         8: 27.0,
         9: 14.0,
         10: 4.0,
         11: 58.0,
         12: 20.0,
         13: 39.0,
         14: 14.0,
         15: 55.0,
         16: 2.0,
         17: nan,
         18: 31.0,
         19: nan}}


In [ ]:
# 행 조회 예시
df_query = "Retrieve the first row."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
format_parser_output(parser_output)


{'0': {'Age': 22.0,
       'Cabin': nan,
       'Embarked': 'S',
       'Fare': 7.25,
       'Name': 'Braund, '
               'Mr. '
               'Owen '
               'Harris',
       'Parch': 0,
       'PassengerId': 1,
       'Pclass': 3,
       'Sex': 'male',
       'SibSp': 1,
       'Survived': 0,
       'Ticket': 'A/5 '
                 '21171'}}


In [ ]:
# row 0 ~ 4의 평균 나이 (결과 비교용)
print(df["Age"].head().mean())

# row 0 ~ 4의 평균 나이 구하도록 프롬프트 작성.
df_query = "0부터 4까지 행의 평균 나이를 구하세요."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
print(parser_output)


31.2
{'mean': 31.2}


In [ ]:
#출력 파서를 사용하지 않을 경우


# 체인 생성
chain = prompt | model

# row 0 ~ 4의 평균 나이 구하도록 프롬프트 작성.
df_query = "0부터 4까지 행의 평균 나이를 구하세요."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
print(parser_output.content)


mean:Age[0..4]


## DatetimeOutputParser


*   날짜 형식 출력 파서



In [ ]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate

# 날짜 및 시간 출력 파서
output_parser = DatetimeOutputParser()
output_parser.format = "%Y-%m-%d"

# 사용자 질문에 대한 답변 템플릿
template = """Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    },  # 지침을 템플릿에 적용
)

# 프롬프트 내용을 출력
prompt


PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%d'.\n\nExamples: 472-05-12, 1762-02-27, 988-08-12\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:')

In [ ]:
# Chain 생성
chain = prompt | llm | output_parser

# 답변 생성
output = chain.invoke({"question": "Google 이 창업한 연도는?"})

# 결과를 문자열로 변환
output.strftime("%Y-%m-%d")

'1998-09-04'

## EnumOutputParser


*   열거형 출력 파서



In [ ]:
from langchain.output_parsers.enum import EnumOutputParser
from enum import Enum

In [ ]:
class Colors(Enum):
    RED = "빨간색"
    GREEN = "초록색"
    BLUE = "파란색"

In [ ]:
#Colors 확인
selected_color = Colors.RED
print(selected_color.RED)
print(selected_color.RED.name)
print(selected_color.RED.value)

Colors.RED
RED
빨간색


In [ ]:
# EnumOutputParser 인스턴스 생성
parser = EnumOutputParser(enum=Colors)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(
    """다음의 물체는 어떤 색깔인가요?  해당 색깔의 이름을 **단일 단어**로만 출력하세요.

Object: {object}

Instructions: {instructions}"""
    # 파서에서 지시사항 형식을 가져와 부분적으로 적용
).partial(instructions=parser.get_format_instructions())
# 체인 생성
chain = prompt | llm | parser


In [ ]:
response = chain.invoke({"object": "하늘"})  # "하늘" 에 대한 체인 호출 실행
print(response)

Colors.BLUE


## OutputFixingParser

*   출력 수정 파서



In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List


class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(
        description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# 잘못된 형식을 일부러 입력
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

##올바른 형식(json은 작은 따옴표를 허용하지 않음)
#misformatted = '{"name": "Tom Hanks", "film_names": ["Forrest Gump"]}'

# 잘못된 형식으로 입력된 데이터를 파싱하려고 시도
parser.parse(misformatted)

# 오류 출력

Actor(name='Tom Hanks', film_names=['Forrest Gump'])

In [ ]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

In [ ]:
# 잘못된 형식의 출력
misformatted

"{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

In [ ]:
# OutputFixingParser 를 사용하여 잘못된 형식의 출력을 파싱
actor = new_parser.parse(misformatted)

In [ ]:
# 파싱된 결과
actor

Actor(name='Tom Hanks', film_names=['Forrest Gump'])